In [1]:
import numpy as np
import  pandas as pd
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier


cd=pd.read_csv("telecom.csv")
cdf=pd.DataFrame(cd)
list1 = [ 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                'TechSupport','StreamingTV', 'StreamingMovies']
for i in list1:
    cdf[i] = cdf[i].replace({'No internet service':'No'})
cdf['MultipleLines'] = cdf['MultipleLines'].replace({'No phone service':'No'})

list2=[ 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection','Churn',
                'TechSupport','StreamingTV', 'StreamingMovies','MultipleLines','Partner','Dependents','PaperlessBilling','PhoneService']
for i in list2:
    cdf[i]= cdf[i].replace({'Yes':1,'No':0})
gen=pd.get_dummies(cdf["gender"],drop_first = True)
cdf=pd.concat((cdf,gen),axis=1)
cdf.drop(['gender'],axis=1,inplace=True)
y=pd.get_dummies(cdf["InternetService"])
y.drop(['No'],axis=1,inplace=True)
cdf.drop(['InternetService'],axis=1,inplace=True)
cdf=pd.concat((cdf,y),axis=1)
x=pd.get_dummies(cdf["Contract"])
cdf.drop(["Contract"],axis=1,inplace=True)
cdf=pd.concat((cdf,x),axis=1)
cdf.drop(["PaymentMethod"],axis=1,inplace=True)
cdf.drop(["customerID"],axis=1,inplace=True)
cdf['tenure']=cdf['tenure'].replace({0:0.1})
ten=np.log(cdf['tenure'])
cdf.drop(["tenure"],axis=1,inplace=True)
cdf=pd.concat((cdf,ten),axis=1)
cdf['TotalCharges'] =pd.to_numeric(cdf['TotalCharges'], errors ='coerce')
z=cdf.dropna()
index =cdf['TotalCharges'].index[cdf['TotalCharges'].apply(np.isnan)]
c=[]
c.extend(index)

d=cdf.loc[c,["MonthlyCharges"]]


x= z["MonthlyCharges"].values

m=len(x)
x=x.reshape((m,-1))




y= z["TotalCharges"].values
y = y.reshape((m,-1))

regmodel=linear_model.LinearRegression()
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.3,random_state=42)
regmodel.fit(x_train,y_train)
a= regmodel.predict(d)

list1=[]
for i in a:
    list1.extend(i)

r2= regmodel.score(x,y)
print("Accuracy of linear model used for predicting a null values of TotalCharges in the data set : ", r2)

for i in list1:
    cdf['TotalCharges']=(cdf['TotalCharges'].fillna(i,limit= 1))


tc=np.log(cdf['TotalCharges'])
cdf.drop(['TotalCharges'],axis=1,inplace=True)
cdf=pd.concat((cdf,tc),axis=1)
mc=np.log(cdf['MonthlyCharges'])
cdf.drop(['MonthlyCharges'],axis=1,inplace=True)
cdf=pd.concat((cdf,mc),axis=1)

cdf=cdf.abs()

from sklearn.model_selection import train_test_split
x=cdf.drop('Churn',axis=1)
y=cdf['Churn']


x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.3,random_state=1)


kfold = model_selection.KFold(n_splits=10, random_state =42)

Accuracy of linear model used for predicting a null values of TotalCharges in the data set :  0.4238558780398537


In [2]:
# Naive bayes

label=list(y_train)
l=x_train
model=GaussianNB()
model.fit(l,label)
predicted=model.predict(x_test) 
print(" Accuracy of Naive bayes model : ",accuracy_score(y_test,predicted)*100)
print()
print("Confusion matrix:")
print(confusion_matrix(y_test,predicted))
print()
print("F1 score of Naive bayes model  :  ",f1_score(y_test,predicted)*100)
print()
results=model_selection.cross_val_score(model,x,y,cv=kfold)*100
print("cross validation of navie bayes model:  ",results)
print()
print("Mean of the result of the cross validation: ",results.mean())


 Accuracy of Naive bayes model :  74.06530998580217

Confusion matrix:
[[1124  461]
 [  87  441]]

F1 score of Naive bayes model  :   61.678321678321666

cross validation of navie bayes model:   [73.19148936 73.33333333 75.17730496 74.43181818 72.15909091 70.45454545
 71.875      73.15340909 71.875      73.15340909]

Mean of the result of the cross validation:  72.88044003868472


In [3]:
#bagging classifier using naive bayes!
cart=GaussianNB()
num_tree =100
model = BaggingClassifier(base_estimator=cart, n_estimators= num_tree, random_state=7)

model.fit(x_train,y_train)
y_ped=model.predict(x_test)

print("Accuracy of the model : ",accuracy_score(y_test,y_ped)*100)
print()
print("confusion Matrix: ")
print(confusion_matrix(y_test,y_ped))
print()
print("F1_score of the model : ", f1_score(y_test,y_ped)*100)
print()
results=model_selection.cross_val_score(model,x,y,cv=kfold)*100
print()
print("Ensemble learner model of type bagging: ")
print()
print("list of accuracy: ",results)
print()
print("Accuracy: of bagging classifier using Decision tree model: ", results.mean())
print()

Accuracy of the model :  74.3019403691434

confusion Matrix: 
[[1129  456]
 [  87  441]]

F1_score of the model :  61.89473684210526


Ensemble learner model of type bagging: 

list of accuracy:  [73.19148936 73.4751773  75.31914894 74.43181818 72.30113636 70.59659091
 71.875      73.4375     72.15909091 73.29545455]

Accuracy: of bagging classifier using Decision tree model:  73.00824065119276

